In [1]:
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np
import pandas as pd
import seaborn as sns


In [22]:
titles = []
years = []
ratings = []
genres = []
runtimes = []
imdb_ratings = []
imdb_ratings_standardized = []
text_muted = []
director = []
stars = []
metascores = []
votes = []


In [12]:
pages = np.arange(1, 9900, 250)
print(pages)


[   1  251  501  751 1001 1251 1501 1751 2001 2251 2501 2751 3001 3251
 3501 3751 4001 4251 4501 4751 5001 5251 5501 5751 6001 6251 6501 6751
 7001 7251 7501 7751 8001 8251 8501 8751 9001 9251 9501 9751]


In [27]:
# for pages in np.arange(1,9900, 250):
for pages in [1]:

    if pages == 1:
        response = get(
            'https://www.imdb.com/search/title/?title_type=feature&num_votes=7500,&languages=en&sort=user_rating,desc&count=250&ref_=adv_prv')
    else:
        response = get(
            'https://www.imdb.com/search/title/?title_type=feature&num_votes=7500,&languages=en&sort=user_rating,desc&count=250&start='+str(pages)+'&ref_=adv_nxt')
    sleep(randint(8, 15))
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(
            requests, response.status_code))
    page_html = BeautifulSoup(response.text, 'html.parser')

    movie_containers = page_html.find_all(
        'div', class_='lister-item mode-advanced')
    for container in movie_containers:
        if container.find('div', class_='item-content') is not None:
            print('yes')

             # title
            title = container.h3.a.text
            titles.append(title)

            if container.h3.find('span', class_='lister-item-year text-muted unbold') is not None:

                    # year released
                    # remove the parentheses around the year and make it an integer
                    year = container.h3.find(
                        'span', class_='lister-item-year text-muted unbold').text
                    years.append(year)

              else:
                    # each of the additional if clauses are to handle type None data, replacing it with an empty string so the arrays are of the same length at the end of the scraping
                    years.append(None)

                if container.p.find('span', class_='certificate') is not None:

                    # rating
                    rating = container.p.find(
                        'span', class_='certificate').text
                    ratings.append(rating)

                else:
                    ratings.append("")

                if container.p.find('span', class_='genre') is not None:

                    # genre
                    genre = container.p.find('span', class_='genre').text.replace("\n", "").rstrip().split(',')  # remove the whitespace character, strip, and split to create an array of genres
                    genres.append(genre)

                else:
                    genres.append("")

                if container.p.find('span', class_='runtime') is not None:

                    # runtime
                    time = int(container.p.find('span', class_='runtime').text.replace(" min", ""))  # remove the minute word from the runtime and make it an integer
                    runtimes.append(time)

                else:
                    runtimes.append(None)

                if float(container.strong.text) is not None:

                    # IMDB ratings
                    # non-standardized variable
                    imdb = float(container.strong.text)
                    imdb_ratings.append(imdb)

                else:
                    imdb_ratings.append(None)

                if container.find('span', class_='metascore').text is not None:

                    # Metascore
                    m_score = int(container.find('span', class_='metascore').text)  # make it an integer
                    metascores.append(m_score)

                else:
                    metascores.append(None)

                if container.find('span', attrs={'name': 'nv'})['data-value'] is not None:

                    # Number of votes
                    vote = int(container.find('span', attrs={'name': 'nv'})['data-value'])
                    votes.append(vote)

                else:
                    votes.append(None)


In [28]:
sci_fi_df = pd.DataFrame({'movie': titles,
                          'year': years,
                          'rating': ratings,
                          'genre': genres,
                          'runtime_min': runtimes,
                          'imdb': imdb_ratings,
                          'metascore': metascores,
                          'votes': votes}
                         )

# two more data transformations after scraping
sci_fi_df.loc[:, 'year'] = sci_fi_df['year'].str[-5:-1]
# Drop 'ovie' bug
# Make year an int
sci_fi_df['n_imdb'] = sci_fi_df['imdb'] * 10
# One small issue with the scrape on these two movies so just dropping those ones.
final_df = sci_fi_df.loc[sci_fi_df['year'] != 'ovie']
final_df.loc[:, 'year'] = pd.to_numeric(final_df['year'])
final_df


AttributeError: Can only use .str accessor with string values!